# Improved Bound Constraint differing across domain

We are interested still in protecting just the outputs. Reiterating the previous paper, we need to find:

$$||f_D(x_*) - f_{D'}(x_*)||^2_H$$

We can express this difference as:

$$f_D(x_*) - f_{D'}(x_*) = \sum_{i=1}^n k(x_*,x_i) \; (\alpha_i - \alpha_i')$$

We also noted that 
$$\mathbf{\alpha} - \mathbf{\alpha}' = K^{-1} (\mathbf{y}-\mathbf{y}')$$

The change we propose at this point is that, rather than just put a bound on the values of $k(\cdot,\cdot)$ we calculate these values for every test point.

So rather than just accept that $-1<k(\cdot,\cdot)<1$ we actually input the value of this function. 

Thus the bound is:

$$\text{max}_j\; \text{abs}\left( \sum_{i=1}^n { k(x_*,x_i) \; (K^{-1})_{ij} } \right) \Delta_y$$

So we need to calculate the kernel value from a test point $x_*$ to each training point $x_i$ (shape, transposed, 1xN). Such a vector will then be multiplied by $K^{-1}$ (NxN) such that each column is multiplied element wise with the $k$s. The resulting 1xN vector is the result of whether the differing element of $\mathbf{y}$ lies at that index. We find the absolute change, as that's what we want to protect. Find the largest of these, and then scale by the largest change in y.

$$\text{max} \; \left[ \text{abs} \left(\mathbf{k}_*^{\top} \mathbf{K}^{-1} \right) \right] \Delta_y$$

For speed we can consider the matrix of covariance between all test and training points $\mathbf{K}_*$, we want the maximum of each row:

$$\text{max}_{by\;row} \; \left[ \text{abs} \left(\mathbf{K}_*^{\top} \mathbf{K}^{-1} \right) \right] \Delta_y$$